In [1]:
import copy

import compound_freecad as FC_Cmp
import core_freecad_ops as FC_Ops
import core_freecad_commands as FC_Cmd

import compound_cameo as MD_Cmp
import core_cameo_ops as MD_Ops
import core_cameo_commands as MD_Cmd

import recipe_oop as RC_OOP
import recipe_systems as RC_Sys

import recipe_reverse_code as RC_RC

import uuid

from docx import *

import javalang

# Section 1 - Load up Base Document and Source Code

We start this example by looking at a document written by hand on a technical topic.

In [2]:
document = Document('Literate Modeling OpenMBEE Sync Technology.docx')

In [3]:
document.paragraphs[1].text

'The data synchronization technology in OpenMBEE is built to bring the model viewing and documentation apparatus into a common state with model authoring tools. The most developed version of this synchronization is built within the Cameo MDK.'

In [4]:
document.paragraphs[1].runs[3].text

' is built to bring the model viewing and documentation apparatus into a common state with model authoring tools. The most developed version of this synchronization is built within the Cameo MDK.'

In [5]:
document.paragraphs[1].runs[1].text

'technology in '

Pull in the source code to cross-examine and reverse engineer. Note that the JAR file can be brought into the MagicDraw Classpath via the properties file and experimented with directly.

In [6]:
source_base = "C:\\Users\\Bjorn\\Documents\\Git\\mdk\\src\\main\\java\\gov\\nasa\\jpl\\mbee\\mdk\\emf\\"
emfimporter_file = open(source_base + "EMFImporter.java", "r")

In [7]:
emfimporter_source = emfimporter_file.readlines()
emfimporter_file.close()

In [8]:
emfimporter_source[365:384]

['                (objectNode, eStructuralFeature, project, strict, element) -> {\n',
 '                    JsonNode jsonNode = objectNode.get(MDKConstants.ID_KEY);\n',
 '                    if (jsonNode == null || !jsonNode.isTextual()) {\n',
 '                            /*if (strict) {\n',
 '                                throw new ImportException(element, objectNode, "Element JSON has missing/malformed ID.");\n',
 '                            }\n',
 '                            return null;*/\n',
 '                        return element;\n',
 '                    }\n',
 '                    try {\n',
 '                        boolean initialCanResetIDForObject = project.getCounter().canResetIDForObject();\n',
 '                        project.getCounter().setCanResetIDForObject(true);\n',
 '                        UNCHECKED_SET_E_STRUCTURAL_FEATURE_FUNCTION.apply(jsonNode.asText(), element.eClass().getEIDAttribute(), element);\n',
 '                        project.getCounter().set

# Section 2 - BulkImporter and EMFImport Tear-Down

In this area, the Merrimac recipes are put together to walk through major sections of the core code for the Import side of the discussion.

## Section 2.1 - Libraries and Common Elements

Make call-backs to core library elements (e.g., basic value types) here.

In [9]:
library_element_ids = \
    {
        'Integer ValueType': '\"_16_5_1_12c903cb_1245415335546_8641_4088\"',
        'Boolean ValueType': '\"_16_5_1_12c903cb_1245415335546_39033_4086\"',
        'String ValueType': '\"_16_5_1_12c903cb_1245415335546_479030_4092\"'
    }
library_element_vars = \
    {
        'Integer ValueType': 'integerVal',
        'Boolean ValueType': 'booleanVal',
        'String ValueType': 'stringVal'
    }

lib_strings = []

for key, val in library_element_ids.items():
    lib_strings.append(
        library_element_vars[key] + ' = \n' +
            '    com.nomagic.magicdraw.core.Application.getInstance().getProject().getElementByID(' +
        val + ');\n'
    )

boolean_obj = RC_OOP.T_Library_Class()
boolean_obj.reference = library_element_vars['Boolean ValueType']
boolean_obj.name = 'boolean'

## Section 2.2 - Major Classes for Importers

Call out the major classes to be described in the document and prototyped

### Section 2.2.1 - Changelog and Associated Classes

In [10]:
changelog_description = 'The Changelog is a class designed to track and aggregate ' + \
'changes discovered between model elements stored within different engineering repositories.'

change_description = 'The Change class is a simple class meant to catalog ' + \
' individual changes as the Changelog is built up on model processing.'

In [11]:
importer_objs = []

changelog_change_class = RC_OOP.T_Class(0)
changelog_change_class.name = 'Change'

changelog_changetype_class = RC_OOP.T_Class(0)
changelog_changetype_class.name = 'ChangeType'

importer_objs.append(changelog_changetype_class)
importer_objs.append(changelog_change_class)

## Section 2.3 - Major Data on which to Operate

Call out the major classes and data types upon which this technology will operate.

### Section 2.3.1 - OpenMBEE and Cameo Data

Classes from the SysML area.

In [12]:
importer_data_objs = []

model_element_class = RC_OOP.T_Class(0)
model_element_class.name = 'Element'

project_class = RC_OOP.T_Class(0)
project_class.name = 'Project'

progress_status_class = RC_OOP.T_Class(0)
progress_status_class.name = 'ProgressStatus'

importer_data_objs.append(model_element_class)
importer_data_objs.append(project_class)
importer_data_objs.append(progress_status_class)

### Section 2.3.2 - Other technologies

Classes from related to technologies.

In [13]:
json_data_objs = []

objectnode_class = RC_OOP.T_Class(0)
objectnode_class.name = 'ObjectNode'

java_stream = RC_OOP.T_Class(0)
java_stream.name = 'java.util.stream.Stream'
java_stream.meta = 'InterfaceBlock'

java_list = RC_OOP.T_Class(0)
java_list.name = 'java.util.List'
java_list.meta = 'InterfaceBlock'

java_predicate = RC_OOP.T_Class(0)
java_predicate.name = 'Predicate'
java_predicate.attributes_dict.update({'method': ''})
java_predicate.att_types_dict.update({'method': library_element_vars['String ValueType']})
java_predicate.att_littypes_dict.update({'method': 'String'})

json_data_objs.append(objectnode_class)
json_data_objs.append(java_stream)
json_data_objs.append(java_list)
json_data_objs.append(java_predicate)

In [14]:
emf_lib_classes = []

estructuralfeature_class = RC_OOP.T_Class(0)
estructuralfeature_class.name = 'EStructuralFeature'

emf_lib_classes.append(estructuralfeature_class)

Adding relevant signatures for later use.

In [15]:
stream_filter = RC_OOP.T_Method_Signature()
stream_filter.name = 'filter'
stream_filter.owning_class = java_stream

stream_map = RC_OOP.T_Method_Signature()
stream_map.name = 'map'
stream_map.owning_class = java_stream

list_stream = RC_OOP.T_Method_Signature()
list_stream.name = 'stream'
list_stream.owning_class = java_list

java_stream.methods.append(stream_filter)
java_stream.methods.append(stream_map)
java_stream.methods.append(list_stream)

## Section 2.4 - Make References from Data to Importer Classes

Show fields where operated data are connected to the importers

In [16]:
changelog_change_class.ref_types_dict.update(
    {
        'changed': model_element_class.generate_cameo_handle(),
        'type': changelog_changetype_class.generate_cameo_handle()
    }
)

## Section 2.5 - Active Sections of Importer Code

This section is for breaking down the larger methods of the importer down into chunks that can be referenced inside the activity model for the algorithm.

### Section 2.5.1 - EMFBulkImporter

Focus on the major steps within the EMF Bulk Importer (import in bulk to an EMF-based in-memory model).

Start with the outline of the apply method under the bulk importer.

In [17]:
#apply on EMFBulkImporter, line 62
importer_code_blocks = []

bulk_importer_apply_op = RC_OOP.T_Method_Signature()
bulk_importer_apply_op.name = 'apply'

bulk_importer_apply_op_objectNodes = RC_OOP.T_Code_Input()
bulk_importer_apply_op_objectNodes.name = 'objectNodes'
bulk_importer_apply_op_objectNodes.single_item = False
bulk_importer_apply_op_objectNodes.para_type = objectnode_class

bulk_importer_apply_op_objectNodes_init = RC_OOP.T_Object_State()
bulk_importer_apply_op_objectNodes_init.working_name = 'objectNodes'

bulk_importer_apply_op_project = RC_OOP.T_Code_Input()
bulk_importer_apply_op_project.name = 'project'
bulk_importer_apply_op_project.single_item = True
bulk_importer_apply_op_project.para_type = project_class

bulk_importer_apply_op_project_init = RC_OOP.T_Object_State()
bulk_importer_apply_op_project_init.working_name = 'project'

bulk_importer_apply_op_progess = RC_OOP.T_Code_Input()
bulk_importer_apply_op_progess.name = 'progressStatus'
bulk_importer_apply_op_progess.single_item = True
bulk_importer_apply_op_progess.para_type = progress_status_class

bulk_importer_apply_op_progess_init = RC_OOP.T_Object_State()
bulk_importer_apply_op_progess_init.working_name = 'progressStatus'

bulk_importer_apply_op.inputs.append(bulk_importer_apply_op_objectNodes)
bulk_importer_apply_op.inputs.append(bulk_importer_apply_op_project)
bulk_importer_apply_op.inputs.append(bulk_importer_apply_op_progess)

bulk_importer_apply_def = RC_OOP.T_Code_Block()
bulk_importer_apply_def.name = 'BulkImporter apply'

bulk_importer_apply_def.passed_parameters.append(bulk_importer_apply_op_objectNodes)
bulk_importer_apply_def.passed_parameters.append(bulk_importer_apply_op_project)
bulk_importer_apply_def.passed_parameters.append(bulk_importer_apply_op_progess)

bulk_importer_apply_def.passed_parameter_state_dict.update(
    {
        bulk_importer_apply_op_objectNodes.name : bulk_importer_apply_op_objectNodes_init
    }
)
bulk_importer_apply_def.passed_parameter_state_dict.update(
    {
        bulk_importer_apply_op_project.name : bulk_importer_apply_op_project_init
    }
)
bulk_importer_apply_def.passed_parameter_state_dict.update(
    {
        bulk_importer_apply_op_progess.name : bulk_importer_apply_op_progess_init
    }
)

bulk_importer_apply_op.implementation = bulk_importer_apply_def

importer_code_blocks.append(bulk_importer_apply_def)

In [18]:
# capture lines 80 through 112
bulk_importer_apply_start = RC_OOP.T_Code_Block()
bulk_importer_apply_start.name = 'BulkImporter apply startup'

# capture lines 113 through 128
bulk_importer_apply_while_start = RC_OOP.T_Code_Block()
bulk_importer_apply_while_start.name = 'BulkImporter bulkImport session start'

# capture lines 129 through 166
bulk_importer_apply_att1 = RC_OOP.T_Code_Block()
bulk_importer_apply_att1.name = 'BulkImporter bulkImport attempt 1'

# capture lines 167 through 200
bulk_importer_apply_att1_5 = RC_OOP.T_Code_Block()
bulk_importer_apply_att1_5.name = 'BulkImporter bulkImport attempt 1.5'

# capture lines 201 through 238
bulk_importer_apply_att2 = RC_OOP.T_Code_Block()
bulk_importer_apply_att2.name = 'BulkImporter bulkImport attempt 2'

### Section 2.5.2 - EMFImporter

Focus on the major steps within the EMF Importer (import a single element).

In [19]:
importer_convert1 = RC_OOP.T_Code_Block()
importer_convert1.name = 'EMFImport find import function for structural feature'

Line 71 to 73 is quite complicated:

ImportFunction function = getEStructuralFeatureOverrides().stream().filter(override -> override.getPredicate()
                        .test(objectNode, eStructuralFeature, project, strict, finalElement)).map(EStructuralFeatureOverride::getFunction)
                        .findAny().orElse(DEFAULT_E_STRUCTURAL_FEATURE_FUNCTION);

Capture EStructuralOverride and its methods.

This is the generic ImportFunction.

In [20]:
import_fun = RC_OOP.T_Class(0)
import_fun.name = 'ImportFunction'
import_fun.meta = 'InterfaceBlock'

import_pred = RC_OOP.T_Class(0)
import_pred.name = 'ImportPredicate'
import_pred.meta = 'InterfaceBlock'
import_pred.general_types.append(java_predicate)

import_pred_test = RC_OOP.T_Method_Signature()
import_pred_test.name = 'test'
import_pred_test.owning_class = import_pred

import_pred.methods.append(import_pred_test)

# add the inputs and returns to match the signature
# boolean test(ObjectNode objectNode, EStructuralFeature eStructuralFeature, Project project, boolean strict, Element element)

import_pred_test_p1 = RC_OOP.T_Code_Input()
import_pred_test_p1.name = 'objectNode'
import_pred_test_p1.single_item = True
import_pred_test_p1.para_type = objectnode_class

import_pred_test_p2 = RC_OOP.T_Code_Input()
import_pred_test_p2.name = 'eStructuralFeature'
import_pred_test_p2.single_item = True
import_pred_test_p2.para_type = estructuralfeature_class

import_pred_test_p3 = RC_OOP.T_Code_Input()
import_pred_test_p3.name = 'project'
import_pred_test_p3.single_item = True
import_pred_test_p3.para_type = project_class

import_pred_test_p4 = RC_OOP.T_Code_Input()
import_pred_test_p4.name = 'strict'
import_pred_test_p4.single_item = True
import_pred_test_p4.para_type = library_element_vars['Boolean ValueType']

import_pred_test_p5 = RC_OOP.T_Code_Input()
import_pred_test_p5.name = 'element'
import_pred_test_p5.single_item = True
import_pred_test_p5.para_type = project_class

import_pred_test.inputs.append(import_pred_test_p1)
import_pred_test.inputs.append(import_pred_test_p2)
import_pred_test.inputs.append(import_pred_test_p3)
import_pred_test.inputs.append(import_pred_test_p4)
import_pred_test.inputs.append(import_pred_test_p5)

Let's try some fun with parsing and ASTs to more quickly build the model:

In [21]:
trial_string = ''
for line in emfimporter_source[488:491]:
    trial_string = trial_string + line
trial_string

'    interface ImportPredicate {\n        boolean test(ObjectNode objectNode, EStructuralFeature eStructuralFeature, Project project, boolean strict, Element element);\n    }\n'

In [22]:
type_list = []
type_list.append(boolean_obj)
type_list.extend(json_data_objs)
type_list.extend(emf_lib_classes)
type_list.append(project_class)
type_list.append(model_element_class)

sig_dict = RC_RC.oop_objects_from_method_sig(
    signature=trial_string,
    types=type_list
)

Signature tree is CompilationUnit(imports=[], package=None, types=[InterfaceDeclaration(annotations=[], body=[MethodDeclaration(annotations=[], body=None, documentation=None, modifiers=set(), name=test, parameters=[FormalParameter(annotations=[], modifiers=set(), name=objectNode, type=ReferenceType(arguments=None, dimensions=[], name=ObjectNode, sub_type=None), varargs=False), FormalParameter(annotations=[], modifiers=set(), name=eStructuralFeature, type=ReferenceType(arguments=None, dimensions=[], name=EStructuralFeature, sub_type=None), varargs=False), FormalParameter(annotations=[], modifiers=set(), name=project, type=ReferenceType(arguments=None, dimensions=[], name=Project, sub_type=None), varargs=False), FormalParameter(annotations=[], modifiers=set(), name=strict, type=BasicType(dimensions=[], name=boolean), varargs=False), FormalParameter(annotations=[], modifiers=set(), name=element, type=ReferenceType(arguments=None, dimensions=[], name=Element, sub_type=None), varargs=False)

In [24]:
sig_dict

{'test': <recipe_oop.T_Method_Signature at 0x25e717b0f28>}

In [29]:
[inp.para_type for inp in sig_dict['test'].inputs]

Capture singletons for specific predicates developed for the bulk importer.

In [ ]:
id_pred = RC_OOP.T_Class(0)
id_pred.name = 'EStructuralFeatureOverride.ID Predicate Singleton'
id_pred.general_types.append(import_pred)
id_pred.attributes_dict.update(
    {
        'method': '\"(objectNode, eStructuralFeature, project, strict, element) -> eStructuralFeature == element.eClass().getEIDAttribute()\"'
    }
)
id_pred.att_types_dict.update({'method': library_element_vars['String ValueType']})
id_pred.att_littypes_dict.update({'method': 'String'})

owner_pred = RC_OOP.T_Class(0)
owner_pred.name = 'EStructuralFeatureOverride.OWNER Predicate Singleton'
owner_pred.general_types.append(import_pred)
owner_pred.attributes_dict.update(
    {
        'method': '\"(objectNode, eStructuralFeature, project, strict, element) -> UMLPackage.Literals.ELEMENT__OWNER == eStructuralFeature\"'
    }
)
owner_pred.att_types_dict.update({'method': library_element_vars['String ValueType']})
owner_pred.att_littypes_dict.update({'method': 'String'})

Capture specific functions within the bulk importer.

In [ ]:
id_fun_string = ''
for indx, line in enumerate(emfimporter_source[365:384]):
    id_fun_string = id_fun_string + '\"' + line[0:-1].replace('\"', '\\"')
    if indx + 1 < len(emfimporter_source[365:384]):
        id_fun_string = id_fun_string + '\" + \n'
    else:
        id_fun_string = id_fun_string + '\"\n'

id_fun = RC_OOP.T_Class(0)
id_fun.name = 'EStructuralFeatureOverride.ID Function Singleton'
id_fun.general_types.append(import_fun)
id_fun.attributes_dict.update(
    {
        'method': id_fun_string
    }
)
id_fun.att_types_dict.update({'method': library_element_vars['String ValueType']})
id_fun.att_littypes_dict.update({'method': 'String'})

owner_fun_string = ''
for indx, line in enumerate(emfimporter_source[406:456]):
    owner_fun_string = owner_fun_string + '\"' + line[0:-1].replace('\"', '\\"')
    if indx + 1 < len(emfimporter_source[406:456]):
        owner_fun_string = owner_fun_string + '\" + \n'
    else:
        owner_fun_string = owner_fun_string + '\"\n'

owner_fun = RC_OOP.T_Class(0)
owner_fun.name = 'EStructuralFeatureOverride.OWNER Function Singleton'
owner_fun.general_types.append(import_fun)
owner_fun.attributes_dict.update(
    {
        'method': owner_fun_string
    }
)
owner_fun.att_types_dict.update({'method': library_element_vars['String ValueType']})
owner_fun.att_littypes_dict.update({'method': 'String'})

## Section 2.6 - Importer and EMF Classes

Capture the structure of the importer codes

In [ ]:
emf_classes = []

emf_bulkimporter_class = RC_OOP.T_Class(0)
emf_bulkimporter_class.name = 'EMFBulkImporter'
emf_bulkimporter_class.methods.append(bulk_importer_apply_op)

bulk_importer_apply_op.owning_class = emf_bulkimporter_class

emf_classes.append(emf_bulkimporter_class)
emf_classes.append(import_fun)
emf_classes.append(import_pred)
emf_classes.append(id_pred)
emf_classes.append(owner_pred)
emf_classes.append(id_fun)
emf_classes.append(owner_fun)

emf_bulkimporter_class.methods[0].implementation

# Section X - Generate Cameo API Calls

## Section X - Create packages for the model

In [ ]:
stringified = []
build_list = MD_Cmd.cameo_build_session_start()

build_list.extend(lib_strings)

for dikt in MD_Cmp.cameo_package_create_dicts(
    name="Base Package",
    order_start=1,
    owner_obj='modelBase'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))

for dikt in MD_Cmp.cameo_package_create_dicts(
    name="MDK Sync Technology",
    order_start=2,
    owner_obj='pkg_id_1'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for dikt in MD_Cmp.cameo_package_create_dicts(
    name="gov.nasa.jpl.mbee.mdk.emf",
    order_start=3,
    owner_obj='pkg_id_2'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for dikt in MD_Cmp.cameo_package_create_dicts(
    name="Cameo Items",
    order_start=4,
    owner_obj='pkg_id_1'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for dikt in MD_Cmp.cameo_package_create_dicts(
    name="Other Java Technology",
    order_start=5,
    owner_obj='pkg_id_1'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for dikt in MD_Cmp.cameo_package_create_dicts(
    name="Eclipse Modeling Framework",
    order_start=6,
    owner_obj='pkg_id_5'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))

## Section X - Create objects

In [ ]:
for obj in importer_data_objs:
    for dikt in obj.generate_for_cameo_abstract('pkg_id_3'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for obj in importer_objs:
    for dikt in obj.generate_for_cameo_abstract('pkg_id_4'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for obj in json_data_objs:
    for dikt in obj.generate_for_cameo_abstract('pkg_id_5'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for obj in importer_code_blocks:
    for dikt in obj.generate_for_cameo_sig_only('pkg_id_3'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for obj in emf_lib_classes:
    for dikt in obj.generate_for_cameo_abstract('pkg_id_5'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for obj in emf_classes:
    for dikt in obj.generate_for_cameo_abstract('pkg_id_3'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))
        
for obj in importer_code_blocks:
    for dikt in obj.generate_for_cameo_internals_only('pkg_id_3'):
        stringified.append(MD_Cmd.cameo_dict_to_string_router(dikt))

In [ ]:
build_list.extend(stringified)
build_list.extend(MD_Cmd.cameo_build_session_end())

f = open("cameo_mdk_tech.groovy", "w")
for build in build_list:
    f.write(build + '\n')
f.close()